# Setting Up Environment

In [2]:
import pandas as pd

In [6]:
predictions = pd.read_csv('mma_data_predictions.csv', index_col = 0)

data = pd.read_csv('mma_data.csv', index_col = 0)
data = data[data.result >= 0]
results_data = data[['fighter_1', 'fighter_2', 'result', 'KO_OVR', 'SUB_OVR']]

odds_data = pd.read_csv('mma_data_odds.csv', index_col = 0)

# Analyzing Accuracy

In [3]:
# Joining predictions to table w/ results and getting result

merged = predictions.merge(results_data, on = ['fighter_1', 'fighter_2'])

# Winner results
merged['Predicted_Result_RF'] = merged.Prediction_RF_Winner.apply(lambda x: 1 if x > 0.5 else 0)
merged['Predicted_Result_GB'] = merged.Prediction_GB_Winner.apply(lambda x: 1 if x > 0.5 else 0)
# merged['Predicted_Resulted_LR'] = merged.Prediction_LR_Winner.apply(lambda x: 1 if x > 0.5 else 0)
merged['Accurate_RF'] = merged.apply(lambda x: 1 if x.result_y == x.Predicted_Result_RF else 0, axis = 1)
merged['Accurate_GB'] = merged.apply(lambda x: 1 if x.result_y == x.Predicted_Result_GB else 0, axis = 1)

# Sub results
merged['Predicted_Sub_RF'] = merged.Prediction_RF_SUB.apply(lambda x: 1 if x > 0.5 else 0)
merged['Predicted_Sub_GB'] = merged.Prediction_GB_SUB.apply(lambda x: 1 if x > 0.5 else 0)
# merged['Predicted_Resulted_LR'] = merged.Prediction_LR_Winner.apply(lambda x: 1 if x > 0.5 else 0)
merged['Accurate_RF_SUB'] = merged.apply(lambda x: 1 if x.SUB_OVR_y == x.Predicted_Sub_RF else 0, axis = 1)
merged['Accurate_GB_SUB'] = merged.apply(lambda x: 1 if x.SUB_OVR_y == x.Predicted_Sub_GB else 0, axis = 1)

# KO Results
merged['Predicted_KO_RF'] = merged.Prediction_RF_KO.apply(lambda x: 1 if x > 0.5 else 0)
merged['Predicted_KO_GB'] = merged.Prediction_GB_KO.apply(lambda x: 1 if x > 0.5 else 0)
# merged['Predicted_Resulted_LR'] = merged.Prediction_LR_Winner.apply(lambda x: 1 if x > 0.5 else 0)
merged['Accurate_RF_KO'] = merged.apply(lambda x: 1 if x.KO_OVR_y == x.Predicted_KO_RF else 0, axis = 1)
merged['Accurate_GB_KO'] = merged.apply(lambda x: 1 if x.KO_OVR_y == x.Predicted_KO_GB else 0, axis = 1)

In [3]:
print(f'Winner accuracy for RF is: {merged.Accurate_RF.mean()*100}%')
print(f'Winner accuracy for GB is: {merged.Accurate_GB.mean()*100}%')
print(f'Sub accuracy for RF is: {merged.Accurate_RF_SUB.mean()*100}%')
print(f'Sub accuracy for GB is: {merged.Accurate_GB_SUB.mean()*100}%')
print(f'KO accuracy for RF is: {merged.Accurate_RF_KO.mean()*100}%')
print(f'KO accuracy for GB is: {merged.Accurate_GB_KO.mean()*100}%')

Winner accuracy for RF is: 55.26315789473685%
Winner accuracy for GB is: 60.526315789473685%
Sub accuracy for RF is: 71.05263157894737%
Sub accuracy for GB is: 73.68421052631578%
KO accuracy for RF is: 42.10526315789473%
KO accuracy for GB is: 50.0%


### Looking At Only Veteran Fights

In [5]:
merged['Fights_1'] = merged.wins_1 + merged.losses_1
merged['Fights_2'] = merged.wins_2 + merged.losses_2

test = merged[(merged.Fights_1 > 15) | (merged.Fights_2 > 15)]

In [7]:
print(f'Winner accuracy for RF is: {test.Accurate_RF.mean()*100}%')
print(f'Winner accuracy for veterans GB is: {test.Accurate_GB.mean()*100}%')
print(f'Sub accuracy for veterans RF is: {test.Accurate_RF_SUB.mean()*100}%')
print(f'Sub accuracy for veterans GB is: {test.Accurate_GB_SUB.mean()*100}%')
print(f'KO accuracy for veterans RF is: {test.Accurate_RF_KO.mean()*100}%')
print(f'KO accuracy for veterans GB is: {test.Accurate_GB_KO.mean()*100}%')

Winner accuracy for RF is: 62.06896551724138%
Winner accuracy for veterans GB is: 65.51724137931035%
Sub accuracy for veterans RF is: 75.86206896551724%
Sub accuracy for veterans GB is: 79.3103448275862%
KO accuracy for veterans RF is: 48.275862068965516%
KO accuracy for veterans GB is: 51.724137931034484%


# Potential Profit

In [26]:
# Getting all the relevant data in one place

odds_data = odds_data[['fighter_1', 'fighter_2', 'Fighter_1_Odds', 'Fighter_2_Odds']]

profit_df = merged.merge(odds_data, on = ['fighter_1', 'fighter_2'])
profit_df.head()

,fighter_1,weight_1,reach_1,age_1,slpm_1,sapm_1,td_avg_1,sub_avg_1,strk_acc_1,strk_def_1,...,Predicted_Sub_RF,Predicted_Sub_GB,Accurate_RF_SUB,Accurate_GB_SUB,Predicted_KO_RF,Predicted_KO_GB,Accurate_RF_KO,Accurate_GB_KO,Fighter_1_Odds,Fighter_2_Odds
0,SeanStrickland,185.0,76.0,32,5.59,4.18,1.09,0.3,40.0,64.0,...,0,0,1,1,0,0,1,1,108.0,-140.0
1,PunaheleSoriano,185.0,11.0,31,3.95,3.42,1.02,0.0,47.0,52.0,...,0,0,1,1,0,0,0,0,-162.0,117.0
2,UmarNurmagomedov,135.0,69.0,27,4.35,0.37,5.02,1.1,70.0,85.0,...,0,0,1,1,0,0,0,0,-1050.0,550.0
3,JavidBasharat,135.0,69.0,28,5.41,2.42,1.70,0.3,56.0,66.0,...,0,0,1,1,1,0,0,1,-410.0,265.0
4,ClaudioRibeiro,185.0,77.0,31,9.60,0.00,0.00,0.0,66.0,0.0,...,0,0,1,1,1,0,1,0,-114.0,-120.0


In [27]:
def rf_bets(row):
    if row.Predicted_Result_RF == 1:
        if (row.Fighter_1_Odds > 100) & (row.Fighter_1_Odds <= 200):
            bet = 50
        elif (row.Fighter_1_Odds > 200):
            bet = 25
        elif (row.Fighter_1_Odds <= 100) & (row.Fighter_1_Odds >= -200):
            bet = 75
        elif (row.Fighter_1_Odds < -200):
            bet = 100
        else:
            bet = 0
    if row.Predicted_Result_RF == 0:
        if (row.Fighter_2_Odds > 100) & (row.Fighter_2_Odds <= 200):
            bet = 50
        elif (row.Fighter_2_Odds > 200):
            bet = 25
        elif (row.Fighter_2_Odds <= 100) & (row.Fighter_2_Odds >= -200):
            bet = 75
        elif (row.Fighter_2_Odds < -200):
            bet = 100
        else:
            bet = 0
    return bet

def calculate_payoff_and_result(row):
    # Calculating Payoff
    if row.Predicted_Result_RF == 1:
        if row.Fighter_1_Odds>0:
            payoff = (row.Fighter_1_Odds/100)*row.Bet
        else:
            payoff = row.Bet/((abs(row.Fighter_1_Odds)/100))
    else:
        if row.Fighter_2_Odds>0:
            payoff = (row.Fighter_2_Odds/100)*row.Bet
        else:
            payoff = row.Bet/((abs(row.Fighter_2_Odds)/100))
    # Calculating Bet Result
    if row.Predicted_Result_RF == row.result:
        bet_result = payoff
    else:
        bet_result = -(row.Bet)
    
    return bet_result
            
profit_df = profit_df[['fighter_1', 'fighter_2', 'Fighter_1_Odds', 'Fighter_2_Odds', 'Predicted_Result_RF', 'result_y']].reset_index(drop = True)
profit_df.columns = ['fighter_1', 'fighter_2', 'Fighter_1_Odds', 'Fighter_2_Odds', 'Predicted_Result_RF', 'result']
profit_df['Bet'] = profit_df.apply(rf_bets, axis = 1)
# profit_df['Bet'] = 100
profit_df['Bet_Result'] = profit_df.apply(calculate_payoff_and_result, axis = 1)

profit_df.Bet_Result.sum()

-659.0888475311815